In [10]:
# The following code backtests some possible "returns" when implementing a strategy using the actual future data to generate signals. We use the convention CrystalBallStrategy to remind us that these strategies are only possible if we had a crystal ball to tell the future. It is our goal to find one that proves to make money and that we can somewhat accurately predict.

from copy import deepcopy

from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.stratanalyzer import returns
from src.features.build_features import BuildFeatures, CrystalBallStrategy

# build our data feeds using MSFT data
base_feed = yahoofeed.Feed()
base_feed.addBarsFromCSV("MSFT", './data/MSFT.csv')
feed =  deepcopy(base_feed)

# calculate the features and labels (we really only need labels)
# we generate two labels that look for observed prices rises
# "1 Day Close", and "15 Days SMA v Close"
features = BuildFeatures(feed, "MSFT")

# attach an analyser to help quantify returns
ret_analyzer = returns.Returns()
features.attachAnalyzer(ret_analyzer)

features.run()
y = features.labels

          Date Ticker   Adj Close Adj Close - Slope15    SMA15  \
498 2020-03-18   MSFT  140.399994            -2.83286  156.089   
499 2020-03-19   MSFT  142.710007            -1.51214  155.057   
500 2020-03-20   MSFT  137.350006            -1.15678  153.413   
501 2020-03-23   MSFT  135.979996            -2.44821  150.959   
502 2020-03-24   MSFT  148.339996            0.518928  149.881   

    Adj Close - SMA15 SMA15 - Slope15 SMA15 - Slope 15 - 2nd Deriv 3  \
498          -15.6887        -2.03795                      0.0654826   
499          -12.3473          -1.838                         0.1265   
500          -16.0633        -1.65614                       0.190905   
501          -14.9793        -1.71209                      0.0629525   
502          -1.54134        -1.63498                      0.0105834   

    Adj Close - EMA15  1 Day Close  15 Days SMA v Close  
498           -14.294          NaN                  NaN  
499           -10.486          NaN                  Na

In [11]:
# As a default we can see what it would look like to have purchased 100 shares of the given ticker and held them. 
print("Final portfolio value: $%.2f" % features.getResult())
print("Cumulative returns: %.2f %%" % (ret_analyzer.getCumulativeReturns()[-1] * 100))

Final portfolio value: $1005773.00
Cumulative returns: 0.58 %


In [12]:
# Now let's backtest a strategy with the training labels we created in BuildFeatures. The strategy is to BUY 100 shares if we predict the price will rise the following day and HOLD until we predict the market will go down the following day, when we SELL 100 shares and wait until we predict another rise.

feed_copy =  deepcopy(base_feed)
one_day_close = CrystalBallStrategy(feed_copy, "MSFT", y.loc[:, '1 Day Close'])
ret_analyzer = returns.Returns()
one_day_close.attachAnalyzer(ret_analyzer)

# Evaluate the strategy with the feed's bars
one_day_close.run()
print("Final portfolio value: $%.2f" % one_day_close.getResult())
print("Cumulative returns: %.2f %%" % (ret_analyzer.getCumulativeReturns()[-1] * 100))

Final portfolio value: $1019423.00
Cumulative returns: 1.94 %


In [13]:
# Backtest another strategy. This strategy is to to BUY 100 shares if we predict the 15-day SMA will be higher than the current stock price in 15 days, and HOLD until we predict the 15-day SMA will be lower than teh current stock price in 15 days, when we SELL 100 shares and wait until we predict another rise.

feed_copy2 =  deepcopy(base_feed)
fifteen_day_sma_v_close = CrystalBallStrategy(feed_copy2, "MSFT", y.loc[:, '15 Days SMA v Close'])
ret_analyzer = returns.Returns()
fifteen_day_sma_v_close.attachAnalyzer(ret_analyzer)

# Evaluate the strategy with the feed's bars
fifteen_day_sma_v_close.run()
print("Final portfolio value: $%.2f" % fifteen_day_sma_v_close.getResult())
print("Cumulative returns: %.2f %%" % (ret_analyzer.getCumulativeReturns()[-1] * 100))

Final portfolio value: $1016922.00
Cumulative returns: 1.69 %
